In [1]:
from Diagnosis import df


Loading processed data from pickle file...


In [2]:
import os
import pandas as pd

# Define the path to the pickle file
pickle_file_path = 'clinic_cleaned.pkl'

# Check if the pickle file already exists
if os.path.exists(pickle_file_path):
    df = pd.read_pickle(pickle_file_path)
    print(f"Pickle file loaded from: {pickle_file_path}")
else:


    column_to_clean = 'Diagnosis'

    df[column_to_clean] = df[column_to_clean].str.replace('?', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('C/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('c/o', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('H/O', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('K/C/O HTN', '', regex=False)
    df[column_to_clean] = df[column_to_clean].replace(r'\\n', ',', regex=True).replace(r'\n', ',', regex=True)
    df[column_to_clean] = df[column_to_clean].str.replace('u/E', '', regex=False)
    df[column_to_clean] = df[column_to_clean].str.replace('U/E', '', regex=False)

    # Deleting column
    column_to_delete = 'Diagonis final'
    if column_to_delete in df.columns:
        df.drop([column_to_delete], axis=1, inplace=True)
    else:
        print(f"Column '{column_to_delete}' not found in the DataFrame.")

    df[column_to_clean] = df[column_to_clean].str.lower()

    # Save the cleaned DataFrame to a pickle file
    df.to_pickle(pickle_file_path)
    print(f"Pickle file saved to: {pickle_file_path}")

# Display the updated DataFrame


Pickle file loaded from: clinic_cleaned.pkl


In [3]:
df['Diagnosis'] = df.apply(
    lambda row: row['Chief Complaint'] if pd.isnull(row['Diagnosis']) and isinstance(row['Chief Complaint'], str) and 
    row['Chief Complaint'].lower() != 'other' and row['Chief Complaint'].lower() != 'other paim' else row['Diagnosis'], 
    axis=1
)


In [4]:
df["Diagnosis"].dropna(inplace=True)


In [8]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process

def clean_and_filter(diagnoses):
    def is_valid_entry(entry):
        return isinstance(entry, str) and entry.strip() and not re.match(r'^[\.\-!@#$%^&*()_+=\[\]{};:\'",<>?/\|\\`~\s]+$', entry)
    
    cleaned = diagnoses[diagnoses.apply(is_valid_entry)].str.strip().dropna().unique()
    return cleaned

def get_best_match(diagnosis, choices, scorer=fuzz.token_sort_ratio):
    if pd.isna(diagnosis) or not diagnosis.strip():
        return diagnosis
    match = process.extractOne(diagnosis, choices, scorer=scorer)
    if match and len(match) >= 2:
        best_match, score = match[:2]  # Take only the first two elements
        return best_match if score >= 80 else diagnosis
    return diagnosis

def process_diagnoses(df, batch_size=10000):
    # Filter data from 2024 onwards
    df = df[df['Date'].dt.year >= 2024]
    
    df_cleaned = df[df['Diagnosis'].apply(lambda x: isinstance(x, str) and x.strip() != '')]
    unique_diagnoses = clean_and_filter(df_cleaned['Diagnosis'])
    
    normalized_diagnoses = []
    for i in range(0, len(df_cleaned), batch_size):
        batch = df_cleaned['Diagnosis'].iloc[i:i+batch_size]
        normalized_batch = batch.apply(lambda x: get_best_match(x, unique_diagnoses))
        normalized_diagnoses.extend(normalized_batch)
    
    df_cleaned['Normalized Diagnosis'] = normalized_diagnoses
    return df_cleaned



In [10]:
result = process_diagnoses(df)


AttributeError: Can only use .dt accessor with datetimelike values

In [12]:
# Assuming 'Diagnosis' is the column with comma-separated values
column_to_split = 'Diagnosis'

# Define a function to process each row
def process_row(row):
    # Check if the 'Diagnosis' column is a string and has comma-separated values
    if isinstance(row[column_to_split], str) and ',' in row[column_to_split]:
        # Split the values by comma
        diagnoses = row[column_to_split].split(',')
        
        # Create new rows for each separated value
        for diagnosis in diagnoses:
            # Copy the rest of the information from the original row
            new_row = row.copy()
            
            # Update the 'Diagnosis' column with the separated value
            new_row[column_to_split] = diagnosis.strip()  # Remove leading/trailing whitespaces
            
            return new_row
    
    else:
        # If no comma-separated values or not a string, return the original row
        return row
    print("Processing done")
# Apply the processing function to each row
result_data = loaded_df.apply(process_row, axis=1)

# Concatenate the resulting DataFrames into a new DataFrame



In [13]:
result_data

,Date,Product,Channel,EMR#,Consultation #,Mobile #,Member #,Type,Status,Patient Gender,...,Chief Complaint,Diagnosis,Advice,REFERRED,Lab Tests,Medicine,Clinic Name,Prescrption json,ICD Code,Dignosis
0,2020-05-15,M-Swasth Doctor - FREE,CashPor,1,1,7571029662,21:01:84:01:25,CALL,DONE,M,...,Other,Other,referred to higher centre,NO,NaN,NO,NaN,"{""pres"":[]}",.,.
1,2021-06-10,M-Swasth Doctor - FREE,CashPor,1,47576,7571029662,21:01:84:01:25,CALL,DONE,M,...,Other,dysmenorrhea and menorrhagia,Advised not miss a single tablet of Ovral l\nD...,NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Tab Ovral L"",""m"":"""",""a"":"""",...",NONE,dysmenorrhea and menorrhagia
2,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48043,7571029662,21:01:84:01:25,CALL,DONE,M,...,Other,weakness with loss of appetite,"Eat healthy food , green vegetables and fruits...",NO,NaN,YES,NaN,"{""pres"":[{""mName"":""Multiprex Cap"",""m"":""1"",""a"":...",NONE,weakness with loss of appetite
3,2021-06-11,M-Swasth Doctor - FREE,CashPor,1,48603,7571029662,21:01:84:01:25,CALL,DONE,M,...,Other,pain,Take nutritious diet\nDo Excercise regularly\n...,NO,CBC,YES,NaN,"{""pres"":[{""mName"":""Paracetamol 500mg Tab"",""m"":...",R50,pain
4,2021-06-16,M-Swasth Doctor - FREE,CashPor,1,52282,7571029662,21:01:84:01:25,CALL,DONE,M,...,Other Pain,gall stones,Referred to nearby hospital for surgical consu...,YES,Ultrasound,NO,NaN,"{""pres"":[]}",K87,gall stones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4990677,2024-05-22,M-Swasth Doctor - PAID,Arohan Financial Services Limited,3804576,5025033,9044949747,240000002515,CALL,DONE,M,...,Skin Problems,vitiligo,Referred to higher center,NO,NaN,NO,NaN,"{""pres"":[]}",L80,vitiligo
4990678,2024-05-22,M-Swasth Doctor - PAID,Bangia Gramin Vikas Bank,3804577,5025034,8637517356,5014010063984,CALL,DONE,F,...,Other Pain,pain abdomen,Avoid spicy/oily food\nTake plenty of oral flu...,NO,NaN,YES,NaN,"{""pres"":[{""dur"":""3"",""a"":"""",""bf"":false,""e"":"""",""...",R10.2,pain abdomen
4990679,2024-05-22,M-Swasth Doctor - FREE,Uttar Banga Kshetriya Garmin Bank,3804578,5025035,9800795009,member1001,CALL,DONE,M,...,Eye Problems,eye pain & redness,"Consult eye specialist\nIf no improvement, the...",NO,NaN,YES,NaN,"{""pres"":[{""dur"":""5"",""isTubeBottle"":true,""a"":""2...",H53,eye pain & redness
4990680,2024-05-22,M-Swasth Combo (Direct),Kamal Fincap Private Limited,3804579,5025036,9828771859,0,CALL,DONE,F,...,Other,headache,Consult eye specialist for glasses\nIf no impr...,NO,"CBC,TFT",YES,NaN,"{""pres"":[{""dur"":""3"",""a"":"""",""bf"":false,""e"":"""",""...",R51,"headache, weakness"
